In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [2]:
d_name = {"苏暮雨": "雨", "苏昌河": "河", "白鹤淮": "淮", "苏喆": "喆", "慕雨墨": "墨", "唐怜月": "月", "慕雪薇": "薇", "慕青羊": "羊"}
l_com = ["雨河", "雨淮", "雨喆", "雨墨", "雨月", "雨薇", "雨羊", "河淮", "淮喆", "墨月", "薇羊"]

df_role = pd.read_excel("暗河传.xlsx", sheet_name="页数", usecols="A:E, K:R, T", na_filter = False).dropna()
df_date = pd.read_excel("暗河传.xlsx", sheet_name="页数", usecols="A:C, U:AE")

#### Check roles

In [3]:
for index, row in df_role.iterrows():
    if row["角色"]:
        for name, aname in d_name.items():
            if name in row["角色"] and name + "（小）" not in row["角色"]:
                if row[aname] != 1:
                    print(row[0], row[1], aname, "should be 1")
            elif row[aname] == 1:
                print(row[0], row[1], aname, "should be 0")

#### Fill in roles

In [ ]:
for index, row in df_role.iterrows():
    if row["组"]:
        for name, aname in d_name.items():
            if name in row["角色"] and name + "（小）" not in row["角色"]:
                df_role.loc[index, aname] = 1
            else:
                df_role.loc[index, aname] = 0
            df_role.loc["总", aname] = df_role[(df_role[aname]==1) | (df_role[aname]==0)][aname].sum()
            if df_role.loc["总", aname] > 0:
                df_role.loc["天", aname] = 1
            else:
                df_role.loc["天", aname] = 0

df_role.to_excel('ahz_fill_in.xlsx', engine='xlsxwriter', index=False)
print("===== fill in complete =====")

#### Count pages

In [ ]:
df_count_page = pd.DataFrame()

pages_t = df_role[(df_role["组"]=="A") | (df_role["组"]=="B")]["页数"].sum()

for name, aname in d_name.items():
    # count "场数"
    df_count_page.loc["总", "场数"] = df_role[(df_role["组"]=="A") | (df_role["组"]=="B")]["页数"].count()
    df_count_page.loc[aname, "场数"] = df_role.value_counts(aname)[1]
    # count "页数"
    df_count_page.loc["总", "页数"] = pages_t
    pages_s = df_role[df_role[aname]==1]["页数"].sum()
    df_count_page.loc[aname, "页数"] = pages_s
    df_count_page.loc[aname, "占比"] = pages_s / pages_t
    # # count "A组"
    pages_a_t = df_count_page.loc["总", "A组"] = df_role[df_role["组"]=="A"]["页数"].sum()
    df_count_page.loc["总", "A%"] = pages_a_t / pages_t
    pages_a = df_role[(df_role["组"]=="A") & df_role[aname]==1]["页数"].sum()
    df_count_page.loc[aname, "A组"] = pages_a
    df_count_page.loc[aname, "A%"] = pages_a / pages_s

for name_ab in l_com:
    # count "场数"
    df_count_page.loc[name_ab[0] + "+" + name_ab[1], "场数"] = df_role[(df_role[name_ab[0]]==1) & (df_role[name_ab[1]]==1)]["页数"].count()
    # count "页数"
    pages_com = df_role[(df_role[name_ab[0]]==1) & (df_role[name_ab[1]]==1)]["页数"].sum()
    df_count_page.loc[name_ab[0] + "+" + name_ab[1], "页数"] = pages_com
    df_count_page.loc[name_ab[0] + "+" + name_ab[1], "占比"] = pages_com / pages_t
    df_count_page.loc[name_ab[0] + "+" + name_ab[1], name_ab[0] + "%"] = pages_com / df_count_page.loc[name_ab[0], "页数"]
    df_count_page.loc[name_ab[0] + "+" + name_ab[1], name_ab[1] + "%"] = pages_com / df_count_page.loc[name_ab[1], "页数"]

df_count_page = df_count_page.style.format({"场数": "{:,.0f}".format, "页数": "{:,.1f}".format, "占比": "{:,.2%}".format, "A组": "{:,.1f}".format, "A%": "{:,.2%}".format, 
                                            "雨%": "{:,.2%}".format, "河%": "{:,.2%}".format, "淮%": "{:,.2%}".format, "喆%": "{:,.2%}".format, "墨%": "{:,.2%}".format, "月%": "{:,.2%}".format, "薇%": "{:,.2%}".format, "羊%": "{:,.2%}".format})

df_count_page

#### Count pages by episode

In [ ]:
pd.options.display.max_columns = None
df_count_page_epi = pd.DataFrame()

for epi in range(1, 41):
    for aname in d_name.values():
        pages_t = df_role[df_role["集"]==epi]["页数"].sum()
        df_count_page_epi.loc["总", epi] = pages_t
        pages_s = df_role[(df_role["集"]==epi) & (df_role[aname]==1)]["页数"].sum()
        df_count_page_epi.loc[aname, epi] = pages_s
        if pages_t:
            df_count_page_epi.loc[aname+"%", epi] = pages_s / pages_t

df_count_page_epi = df_count_page_epi.T
# df_count_page_epi = df_count_page_epi.style.format({"总": "{:,.1f}", "雨": "{:,.1f}".format, "河": "{:,.1f}".format, "淮": "{:,.1f}".format, "喆": "{:,.1f}".format, "墨": "{:,.1f}".format, "月": "{:,.1f}".format, "薇": "{:,.1f}".format, "羊": "{:,.1f}".format,
#     "雨%": "{:,.2%}".format, "河%": "{:,.2%}".format, "淮%": "{:,.2%}".format, "喆%": "{:,.2%}".format, "墨%": "{:,.2%}".format, "月%": "{:,.2%}".format, "薇%": "{:,.2%}".format, "羊%": "{:,.2%}".format})
df_count_page_epi


In [ ]:
matplotlib.rcParams['font.family'] = ['Heiti TC']
plt.figure(figsize=(40, 10))
plt.bar(range(1, 41), df_count_page_epi["总"], color=["#b6cca1"], alpha=0.3)
for aname in d_name.values():
    plt.plot(range(1, 41), df_count_page_epi[aname], "-o", label=aname, alpha=0.6)
plt.xticks(range(1, 41));
plt.xlim(0.2, 40.8)
plt.xlabel("episode")
plt.ylabel("pages")
plt.legend();

In [ ]:
matplotlib.rcParams['font.family'] = ['Heiti TC']
plt.figure(figsize=(40, 10))
plt.bar(range(1, 41), df_count_page_epi["总"], color=["#b6cca1"], alpha=0.3)
plt.plot(range(1, 41), df_count_page_epi["雨"], "-o", label="雨", alpha=0.7)
plt.plot(range(1, 41), df_count_page_epi["河"], "-o", label="河", alpha=0.7)
plt.plot(range(1, 41), df_count_page_epi["淮"], "-o", label="淮", alpha=0.7)
plt.xticks(range(1, 41));
plt.xlim(0.2, 40.8)
plt.xlabel("episode")
plt.ylabel("pages")
plt.legend();

#### Count others pages

In [ ]:
names = ["萧若风", "李寒衣", "李心月", "谢宣", "萧永", "浊清"]

for name in names:
    print(name, df_role[(df_role["组"]=="A") | (df_role["组"]=="B")][df_role["角色"].str.contains(name, na = False)]["页数"].sum())

#### Count dates

In [ ]:
dates = ["拍摄日期", "Unnamed: 21", "Unnamed: 22", "Unnamed: 23", "Unnamed: 24", "Unnamed: 25", "Unnamed: 26", "Unnamed: 27", "Unnamed: 28", "Unnamed: 29", "Unnamed: 30"]

for index, row in df_date.iterrows():
    if not pd.isna(row["拍摄日期"]):
        df_date.loc[index, "天数"] = 0
        for date in dates:
            if not pd.isna(df_date.loc[index, date]):
                df_date.loc[index, "天数"] += 1

df_count_date = df_date[["集", "场", "主要内容", "天数"]]
# df_count_date = df_count_date.fillna(0)
# df_count_date["天数"] = df_count_date["天数"].astype(int)
df_count_date = df_count_date.sort_values(by=["天数"], ascending=False)

df_count_date.to_excel("ahz_count_date.xlsx", index=False)
df_count_date[:30]

#### Export to table

In [11]:
date = "10-28"

##### A

In [ ]:
with open("ahz.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()

# create group for each 8 lines
groups = [lines[i:i+8] for i in range(0, len(lines), 8)]
for group in groups:
    js = group[0]
    cc = group[1]
    qf = group[2]
    nw = group[3]
    zcj = group[4]
    fcj = group[5]
    zynr = group[6]
    ys = group[7]
    group[0:20] = [group[0], group[1], group[6], group[7], "A", group[2], group[3], "\t", group[4], group[5],
                   "\t", "\t", "\t", "\t", "\t", "\t", "\t", "\t", "\t", "\t", date]

# replace '\n' with '\t' and join the group with a newline
result = "\n".join(["\t".join([line.strip() for line in group]) for group in groups])
print(result)

with open('ahz_result.txt', 'w', encoding='utf-8') as file:
    file.write(result)

##### B

In [ ]:
with open("ahz.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()

# create group for each 8 lines
groups = [lines[i:i+8] for i in range(0, len(lines), 8)]
for group in groups:
    js = group[0]
    cc = group[1]
    qf = group[2]
    nw = group[3]
    zcj = group[4]
    fcj = group[5]
    zynr = group[6]
    ys = group[7]
    group[0:20] = [group[0], group[1], group[6], group[7], "B", group[2], group[3], "\t", group[4], group[5],
                   "\t", "\t", "\t", "\t", "\t", "\t", "\t", "\t", "\t", "\t", date]

# replace '\n' with '\t' and join the group with a newline
result = "\n".join(["\t".join([line.strip() for line in group]) for group in groups])
print(result)

with open('ahz_result.txt', 'w', encoding='utf-8') as file:
    file.write(result)